<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети
####Вариант 5:
Просчитать одну итерацию цикла обучения по Δ-правилу однослойной бинарной
неоднородной нейронной сети, состоящей из 2 нейронов и имеющей функции активации: гиперболический тангенс ($k=2$) и пороговую функцию ($Т=0.5$). В качестве обучающей выборки использовать таблицу истинности для операций эквивалентности и конъюнкции (не использовать первую строчку таблицы). Синаптические веса задать случайным образом.

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [4]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.6
#Количество слоёв
layer = 1
#количество нейронов на каждом слое
neurons = 2
#количество входов/выходов соответственно
countX = 2
countY = 2

networkCharact = [layer, neurons, countX, countY, learningRate]

#--------------------------------------------------------------------

#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#--------------------------------------------------------------------

def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

In [16]:
def deltaRule(function, data, level = 1, N = 2, nabla = 0.1):
  wi = [[randomNumber() for i in range(N)] for j in range(N)]         #синаптические веса
  si = [0] * N
  yi = [0] * N
  epsi = [0] * N

  level_i = 0
  k = 0
  while(level_i < level): #цикл по слоям
    print("\tЭпоха №" + str(level_i + 1))
    while(k < len(data)): #цикл по строкам таблицы с данными
      eps = 0
      print("Матрица весов для " + str(k + 1) + "-й итерации:")
      print(pd.DataFrame(wi))
      print("\nДля X = ( " + str(data[data.columns[0]][k]) + ", " + str(data[data.columns[1]][k]) + ' )')

      for i in range(N):
        for j in range(N):
          si[i] += data[data.columns[j]][k] * wi[j][i]

        yi[i] = function(si[i])
        epsi[i] = data[data.columns[N + i]][k] - yi[i]
        eps += epsi[i]**2

      rez_i = pd.DataFrame([si, yi, epsi], index=['S', 'Y расч.', 'EPS'], columns=['neuron ' + str(i+1) for i in range(N)])
      print(rez_i)
      print("\nСКО: " + str(eps) + "\n\n")

      #вычисление весов для следующей итерации:
      for i in range(N):
        for j in range(N):
          wi[j][i] = wi[j][i] - nabla * epsi[i] * data[data.columns[j]][k]

      k += 1
    level_i += 1


  #return 0

In [17]:
table = pd.DataFrame(it.product([0,1], repeat=countX), columns=['X' + str(i + 1) for i in range(countX)])
table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

deltaRule(hyperbolicTangentFunction, table, layer, neurons, learningRate)
#deltaRule(sigmoidFunction, table, layer, neurons, learningRate)
#deltaRule(thresholdFunction, table, layer, neurons, learningRate)

	Эпоха №1
Матрица весов для 1-й итерации:
          0         1
0  0.468062  0.041372
1  0.894114  0.298415

Для X = ( 0, 0 )
         neuron 1  neuron 2
S             0.0       0.0
Y расч.       0.0       0.0
EPS           1.0       0.0

СКО: 1.0


Матрица весов для 2-й итерации:
          0         1
0  0.468062  0.041372
1  0.894114  0.298415

Для X = ( 0, 1 )
         neuron 1  neuron 2
S        0.894114  0.298415
Y расч.  0.419477  0.148110
EPS     -0.419477 -0.148110

СКО: 0.1978975663051355


Матрица весов для 3-й итерации:
          0         1
0  0.468062  0.041372
1  1.145801  0.387281

Для X = ( 1, 0 )
         neuron 1  neuron 2
S        1.362176  0.339787
Y расч.  0.592226  0.168278
EPS     -0.592226 -0.168278

СКО: 0.37904943523971674


Матрица весов для 4-й итерации:
          0         1
0  0.823398  0.142338
1  1.145801  0.387281

Для X = ( 1, 1 )
         neuron 1  neuron 2
S        3.331375  0.869407
Y расч.  0.930979  0.409244
EPS      0.069021  0.590756

СКО: 0.353

In [ ]:
#1-Количество входов и выходов могут быть различны
#2-Реализовать расчёт промежуточных слоёв